In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display
import librosa.display
import numpy as np
import librosa
import tensorflow as tf
import glob

### If you have been save the data, you don't have to preprocessing and save the data

In [2]:
drone_path = '../../data/drone/*.wav'
background_path = '../../data/background-tmp/*.wav'

drone_files = glob.glob(drone_path)
background_files = glob.glob(background_path)

In [3]:
CHUNK_SIZE = 8192
SR = 22050
N_MFCC = 16

In [4]:
def load(files, sr=22050):
    [raw, sr] = librosa.load(files[0], sr=sr)
    for f in files[1:]:
        [array, sr] = librosa.load(f, sr=sr)
        raw = np.hstack((raw, array))
    print(raw.shape)
    return raw

In [5]:
drone_raw = load(drone_files)
background_raw = load(background_files)

(11406619,)
(11127941,)


# Data Processing

In [6]:
def mfcc4(raw, label, chunk_size=8192, window_size=4096, sr=22050, n_mfcc=16, n_frame=16):
    mfcc = np.empty((0, n_mfcc, n_frame))
    y = []
    print(raw.shape)
    for i in range(0, len(raw), chunk_size//2):
        mfcc_slice = librosa.feature.mfcc(raw[i:i+chunk_size], sr=sr, n_mfcc=n_mfcc) #n_mfcc,17
        if mfcc_slice.shape[1] < 17:
            print("small end:", mfcc_slice.shape)
            continue
        mfcc_slice = mfcc_slice[:,:-1]
        mfcc_slice = mfcc_slice.reshape((1, mfcc_slice.shape[0], mfcc_slice.shape[1]))
        mfcc = np.vstack((mfcc, mfcc_slice))
        y.append(label)
    y = np.array(y)
    return mfcc, y

In [7]:
mfcc_drone, y_drone = mfcc4(drone_raw, 1)
mfcc_background, y_background = mfcc4(background_raw, 0)

print(mfcc_drone.shape, y_drone.shape)
print(mfcc_background.shape, y_background.shape)

(11406619,)
small end: (16, 15)
small end: (16, 7)
(11127941,)
small end: (16, 15)
small end: (16, 7)
(2783, 16, 16) (2783,)
(2715, 16, 16) (2715,)


In [8]:
X = np.concatenate((mfcc_drone, mfcc_background), axis=0)
#X = np.concatenate((mfcc_drone), axis=0)
#X = X.reshape(-1, 16,16,1)
y = np.hstack((y_drone, y_background))
#y = np.hstack(y_drone)
print(X.shape, y.shape)

(5498, 16, 16) (5498,)


In [9]:
n_labels = y.shape[0]
n_unique_labels = 2
y_encoded = np.zeros((n_labels, n_unique_labels))
y_encoded[np.arange(n_labels), y] = 1
print(y_encoded.shape)

(5498, 2)


In [10]:
# Split data
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y_encoded, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = model_selection.train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [11]:
print(X_train.shape, X_test.shape)
print(X_val.shape, y_val.shape)
print(y_train.shape, y_test.shape)

(3518, 16, 16) (1100, 16, 16)
(880, 16, 16) (880, 2)
(3518, 2) (1100, 2)


In [12]:
# Save Data
np.save('../../data/X_train', X_train)
np.save('../../data/X_test', X_test)
np.save('../../data/X_val', X_val)
np.save('../../data/y_val', y_val)
np.save('../../data/y_train', y_train)
np.save('../../data/y_test', y_test)

### Until this part

In [13]:
# Load Data
X_train = np.load('../../data/X_train.npy')
X_test = np.load('../../data/X_test.npy')
X_val = np.load('../../data/X_val.npy')
y_val = np.load('../../data/y_val.npy')
y_train = np.load('../../data/y_train.npy')
y_test = np.load('../../data/y_test.npy')

# Experiment 3 - One convolutional layer /w no dropout

##Experiment 3-2
- learning rate 0.005
- pooling stride 1x1
- #filter 1
- best result among every other settings
- cost kept fluctuated during training. (0.8 -> 1.3) -- why is that?

In [14]:
tf.reset_default_graph()

In [15]:
n_mfcc = 16
n_frame = 16
n_classes = 2
n_channels = 1

learning_rate = 0.0002  # 0.005
training_epochs = 200 # 수정해봐

# Layer

In [16]:
X = tf.placeholder(tf.float32, shape=[None,n_mfcc*n_frame*n_channels])
X = tf.reshape(X, [-1, n_mfcc, n_frame, n_channels])
Y = tf.placeholder(tf.float32, shape=[None,n_classes])

conv1 = tf.layers.conv2d(inputs=X, filters=1, kernel_size=[3, 3],
                         padding="SAME", activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],
                                padding="SAME", strides=1)
# dropout넣어야하나
conv2 = tf.layers.conv2d(inputs=pool1, filters=1, kernel_size=[3, 3],
                         padding="SAME", activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2],
                                padding="SAME", strides=1)
# 여기도
flat = tf.reshape(pool2, [-1, 16*16*1])
dense2 = tf.layers.dense(inputs=flat, units=625, activation=tf.nn.relu)
logits = tf.layers.dense(inputs=dense2, units=2)

In [17]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [18]:
Y_pred = tf.contrib.layers.fully_connected(logits,n_classes,activation_fn = None)

In [19]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [20]:
X_train2 = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test2 = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
X_val2 = X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2], 1)

In [21]:
# model save
model_path = '../../model/CNN/cnn_model'
saver = tf.train.Saver()

# Trainning

In [22]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import itertools as it

In [23]:
###########################
batch_size = 32
cost_history = np.empty(shape=[1], dtype=float)
with tf.device("/gpu:0"):
    for epoch in range(training_epochs):#training epoch 500 / batch_size 128 --> acc 90%
        avg_cost = 0
        val_avg_cost =0
        total_batch = int(y_train.shape[0] / batch_size)
        for i in range(0, y_train.shape[0], batch_size):
            feed_dict={X:X_train2[i:i+batch_size,:,:,:], Y:y_train[i:i+batch_size,:]}
            c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
            cost_history = np.append(cost_history,cost)
            avg_cost += c/total_batch 

        y_pred = sess.run(logits, feed_dict={X:X_val2})
        y_pred = sess.run(tf.argmax(y_pred,1))
        y_true = y_val

        y_true = sess.run(tf.argmax(y_true,1))
        print(len(y_pred),end=' ')
        print('Epoch:', '%04d' % (epoch+1), 'cost = ', '{:.9f}'.format(avg_cost), 'val = ','%f' %(accuracy_score(y_true, y_pred)) )
saver.save(sess, model_path)

880 Epoch: 0001 cost =  0.882419200 val =  0.968182
880 Epoch: 0002 cost =  0.196591294 val =  0.969318
880 Epoch: 0003 cost =  0.161796901 val =  0.972727
880 Epoch: 0004 cost =  0.140712265 val =  0.972727
880 Epoch: 0005 cost =  0.127785777 val =  0.976136
880 Epoch: 0006 cost =  0.124499973 val =  0.967045
880 Epoch: 0007 cost =  0.123583314 val =  0.972727
880 Epoch: 0008 cost =  0.107945074 val =  0.973864
880 Epoch: 0009 cost =  0.095082831 val =  0.973864
880 Epoch: 0010 cost =  0.086398893 val =  0.975000
880 Epoch: 0011 cost =  0.075832051 val =  0.970455
880 Epoch: 0012 cost =  0.084189737 val =  0.959091
880 Epoch: 0013 cost =  0.078910422 val =  0.971591
880 Epoch: 0014 cost =  0.070348343 val =  0.960227
880 Epoch: 0015 cost =  0.072706838 val =  0.946591
880 Epoch: 0016 cost =  0.096965130 val =  0.950000
880 Epoch: 0017 cost =  0.221138989 val =  0.981818
880 Epoch: 0018 cost =  0.116964159 val =  0.981818
880 Epoch: 0019 cost =  0.066181420 val =  0.984091
880 Epoch: 0

880 Epoch: 0159 cost =  0.009384704 val =  0.981818
880 Epoch: 0160 cost =  0.003731987 val =  0.985227
880 Epoch: 0161 cost =  0.004610961 val =  0.982955
880 Epoch: 0162 cost =  0.003380247 val =  0.981818
880 Epoch: 0163 cost =  0.003491379 val =  0.984091
880 Epoch: 0164 cost =  0.006443148 val =  0.982955
880 Epoch: 0165 cost =  0.003188341 val =  0.977273
880 Epoch: 0166 cost =  0.003766948 val =  0.981818
880 Epoch: 0167 cost =  0.002870603 val =  0.986364
880 Epoch: 0168 cost =  0.004875846 val =  0.986364
880 Epoch: 0169 cost =  0.003136263 val =  0.977273
880 Epoch: 0170 cost =  0.003302423 val =  0.984091
880 Epoch: 0171 cost =  0.004043895 val =  0.977273
880 Epoch: 0172 cost =  0.002197640 val =  0.981818
880 Epoch: 0173 cost =  0.001704386 val =  0.981818
880 Epoch: 0174 cost =  0.001988056 val =  0.980682
880 Epoch: 0175 cost =  0.004728274 val =  0.982955
880 Epoch: 0176 cost =  0.001258940 val =  0.982955
880 Epoch: 0177 cost =  0.000812147 val =  0.986364
880 Epoch: 0

'../../model/CNN/cnn_model'

## Prediction

In [24]:
y_pred = sess.run(tf.argmax(logits,1),feed_dict={X: X_test2})
y_true = sess.run(tf.argmax(y_test,1))

In [25]:
# Print Result

from sklearn.metrics import precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
print("F-Score:", round(f,3))
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_true, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))

F-Score: 0.987
Accuracy:  0.9872727272727273
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       517
           1       0.99      0.99      0.99       583

   micro avg       0.99      0.99      0.99      1100
   macro avg       0.99      0.99      0.99      1100
weighted avg       0.99      0.99      0.99      1100

[[510   7]
 [  7 576]]
